In [1]:
import nltk
from nltk.stem import WordNetLemmatizer

import re
from bs4 import BeautifulSoup
import spacy
import numpy as np

In [2]:
lemmatizer = WordNetLemmatizer()
print (lemmatizer.lemmatize('working', pos='v'))

work


In [27]:
def load_data(FILE_Path):
    otuput_sent = []
    lemmatizer = WordNetLemmatizer()
    with open(FILE_Path, "r") as f:
        train_tmp = f.read().split("\n\n")[:-1]
        for sample in train_tmp:
            sentence = re.split('\t|\n', sample)[1].replace('\"',"").replace(".","").replace(",","")\
            .replace("?","").replace("!","").replace(")","").replace("(","").replace("\'s","")\
            .replace("\'ve","").replace("\'t","").replace("\'re","").replace("\'d","")\
            .replace("\'ll","").replace("'","").replace(";","").replace(":","")
            answer = re.split('\t|\n', sample)[2]
            soup = BeautifulSoup(sentence,"lxml")

            e1 = str(soup.find('e1'))[4:-5]
            e2 = str(soup.find('e2'))[4:-5]
            word_list = sentence.split(" ")
            e1_check = 0
            e2_check = 0
            for word in word_list:
                if word.endswith("</e1>"):
                    e1_check = 1
                if word.endswith("</e2>"):
                    e2_check = 1
            if e1_check + e2_check != 2:
                print(sentence)
            for word in word_list:
                if "</e1>" in word:
                    a = word
                if "</e2>" in word:
                    b = word
            p1 = word_list.index(a)
            p2 = word_list.index(b)
            sentence = sentence.replace("<e1>","").replace("</e1>","").replace("<e2>","").\
            replace("</e2>","")
            sentence = [lemmatizer.lemmatize(word, pos='v') for word in sentence.split(" ")]
            sentence = " ".join(sentence)
            otuput_sent.append((sentence,e1,e2,p1,p2,answer))
    return otuput_sent

In [28]:
train_instance = load_data("./data/TRAIN_FILE.txt")
print("--------")
test_instance = load_data("./data/TEST_FILE_FULL.txt")

--------


In [30]:
len(test_instance)

2717

In [5]:
train_text = [ele[0] for ele in train_instance]

In [9]:
print(train_text[4783])
train_position[4783]

When thecoil of the relay be at rest not energize the common terminal 30 and the normally close terminal 87a have continuity


(1, 4)

In [6]:
train_position = [(ele[3],ele[4]) for ele in train_instance]

In [40]:
# check p1 p2 order
for a, b in train_position:
    if a > b: 
        print(a,b)

In [43]:
len(train_instance)

8000

### depency parsing

In [45]:
nlp = spacy.load('en_core_web_sm')

In [49]:
train_text = [ele[0] for ele in train_instance]
test_test = [ele[0] for ele in test_instance]

In [50]:
demo_id = 62
demo_sentence = train_text[demo_id]
print(demo_sentence)

doc = nlp(demo_sentence)

for pos, token in enumerate(doc):
    if pos in train_position[demo_id]:
        ancestors = [i for i in token.ancestors]
        print(token.text, token.tag_, token.dep_,
              token.shape_, token.is_stop, token.ent_type,ancestors,
              "lefts",[word.text for word in token.lefts],
              "rights",[word.text for word in token.rights],
              token.n_lefts,  # 2
            token.n_rights  # 1
             )

A great wing of fluorite dragons the greatest concentration of the rather solitary dragons ever known joined the desperate battle with the blues and silvers against Chaos
wing NN nsubj xxxx False 0 [concentration, joined] lefts ['A', 'great'] rights ['of'] 2 1
dragons NNS pobj xxxx False 0 [of, wing, concentration, joined] lefts ['fluorite'] rights [] 1 0


In [104]:
train_mut_ancestors_list = []
train_dep_list = []
for ele in train_instance:
    sentence = ele[0]
    doc = nlp(sentence)
    entity = ele[1:3]
    both_ancestors = []
    both_dep = ""
    for pos, token in enumerate(doc):
        if pos in ele[3:5]:
            ancestors = [i.text for i in token.ancestors]
            both_ancestors.append(ancestors)
            both_dep+=token.dep_
    a = 0
    b = 0
    if entity[0] in both_ancestors[1]:
        a = 1
    elif entity[1] in both_ancestors[0]:
        b = 1
    mut_ancestors = [a,b]
    train_mut_ancestors_list.append(mut_ancestors)
    train_dep_list.append(both_dep)
    
test_mut_ancestors_list = []
test_dep_list = []
for ele in test_instance:
    sentence = ele[0]
    doc = nlp(sentence)
    entity = ele[1:3]
    both_ancestors = []
    both_dep = ""
    for pos, token in enumerate(doc):
        if pos in ele[3:5]:
            ancestors = [i.text for i in token.ancestors]
            both_ancestors.append(ancestors)
            both_dep+=token.dep_
    a = 0
    b = 0
    if entity[0] in both_ancestors[1]:
        a = 1
    elif entity[1] in both_ancestors[0]:
        b = 1
    mut_ancestors = [a,b]
    test_mut_ancestors_list.append(mut_ancestors)
    test_dep_list.append(both_dep)

In [73]:
np.save("./features/train_mut_ancestors_list.npy", train_mut_ancestors_list)
np.save("./features/test_mut_ancestors_list.npy", test_mut_ancestors_list)

In [119]:
a, b = np.unique(train_dep_list, return_counts=True)

In [137]:
a_sorted = a[np.argsort(b)[::-1]]

In [142]:
major_dep = a_sorted[:33]
major_dep

array(['nsubjpobj', 'dobjpobj', 'pobjpobj', 'nsubjdobj', 'nsubjpasspobj',
       'compoundpobj', 'nsubjcompound', 'attrpobj', 'compoundnsubj',
       'compounddobj', 'conjpobj', 'pobjdobj', 'nsubjnsubj', 'nsubjpunct',
       'pobjcompound', 'compoundcompound', 'dobjdobj', 'amodpobj',
       'dobjcompound', 'ROOTpobj', 'nsubjconj', 'nsubjpasscompound',
       'nsubjamod', 'nsubjpassdobj', 'attrdobj', 'punctprep', 'appospobj',
       'pobjconj', 'pobjnsubj', 'compoundattr', 'nsubjnmod', 'pobjattr',
       'amoddobj'], dtype='<U18')

In [139]:
sorted(b,reverse=True)[:33]

[1784,
 928,
 838,
 833,
 688,
 320,
 172,
 151,
 131,
 111,
 110,
 87,
 83,
 65,
 64,
 54,
 50,
 50,
 46,
 46,
 45,
 40,
 39,
 38,
 27,
 27,
 26,
 25,
 25,
 24,
 22,
 20,
 20]

In [143]:
train_dep_list_filter = []
for dep in train_dep_list:
    if dep in major_dep:
        train_dep_list_filter.append(dep)
    else:
        train_dep_list_filter.append("other")

test_dep_list_filter = []
for dep in test_dep_list:
    if dep in major_dep:
        test_dep_list_filter.append(dep)
    else:
        test_dep_list_filter.append("other")

In [146]:
np.unique(test_dep_list_filter, return_counts=True)

(array(['ROOTpobj', 'amoddobj', 'amodpobj', 'appospobj', 'attrdobj',
        'attrpobj', 'compoundattr', 'compoundcompound', 'compounddobj',
        'compoundnsubj', 'compoundpobj', 'conjpobj', 'dobjcompound',
        'dobjdobj', 'dobjpobj', 'nsubjamod', 'nsubjcompound', 'nsubjconj',
        'nsubjdobj', 'nsubjnmod', 'nsubjnsubj', 'nsubjpasscompound',
        'nsubjpassdobj', 'nsubjpasspobj', 'nsubjpobj', 'nsubjpunct',
        'other', 'pobjattr', 'pobjcompound', 'pobjconj', 'pobjdobj',
        'pobjnsubj', 'pobjpobj', 'punctprep'], dtype='<U17'),
 array([ 22,   7,  13,   3,  11,  61,   8,  16,  38,  25, 102,  25,  16,
         22, 311,  12,  45,  22, 246,   8,  25,  22,  15, 239, 648,  27,
        340,  10,  28,   8,  38,   9, 283,  12]))

In [171]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
le = LabelEncoder()
enc = OneHotEncoder(sparse=False)
lb = LabelBinarizer()

In [188]:
# transform string to onehot
train_dep_list_filter = lb.fit_transform(train_dep_list_filter)
test_dep_list_filter = lb.transform(test_dep_list_filter)

In [189]:
np.save("./features/train_dep_list.npy", train_dep_list_filter)
np.save("./features/test_dep_list.npy", test_dep_list_filter)

In [192]:
train_dep_list_filter

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [191]:
test_dep_list_filter

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])